In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# import os
# os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" # this is highly important for totally utilizing your GPU's memory, but it also breaks the profiler's memory breakdown

import numpy as np
import qutip as qt 
from QOGS.optimizer.tf_adam_optimizer import AdamOptimizer
from QOGS.gate_sets import SNAP_disp
from QOGS.optimizer.GateSynthesizer import GateSynthesizer
import matplotlib.pyplot as plt

In [ ]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 40
# We initialize the ECDGateSet here
gate_set_params = {
    'N_cav' : N,
    'N_snap' : 6,
    'beta_scale' : 3.0
}
SNAP_gate_set = SNAP_disp(**gate_set_params)

In [ ]:
# Setup the initial and final states of the X gate. This does not define a unitary, but the optimizer will find one that performs these transfers
x = 1 / np.sqrt(2)

# +Z -> -Z and vice-versa in code and error space
pZ = x * (qt.basis(N, 0) + qt.basis(N, 4))
mZ = qt.basis(N, 2)
pZe = qt.basis(N, 3)
mZe = qt.basis(N, 1)

# |+> -> |+> and |i> -> |-i> in the code space
plus = x * (pZ + mZ)
i_state = x * (pZ + 1j * mZ)
mi_state = x * (pZ - 1j * mZ)

# |+> -> |+> and |i> -> |-i> in the error space
plus_e = x * (pZe + mZe)
i_state_e = x * (pZe + 1j * mZe)
mi_state_e = x * (pZe - 1j * mZe)

inits = [pZ, pZe, mZ, mZe, plus, plus_e]
finals = [mZ, mZe, pZ, pZe, plus, plus_e]

In [ ]:
#the optimization options
synth_params = {
    'gateset' : SNAP_gate_set,
    'N_blocks': 6,
    'N_multistart' : 500, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 500, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'initial_states' : finals, #qubit tensor oscillator, start in |g> |0>
    'target_states' : inits, #end in |e> |target>.
    'name' : 'binomial X gate SNAP', #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [ ]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

In [ ]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

In [ ]:
#can print info, including the best circuit found.
gatesynth.print_info() 

In [ ]:
print(gatesynth.best_fidelity())

In [ ]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
thetas = best_circuit['thetas']
print(best_circuit)